   ### Description
   The Mountain Car MDP is a deterministic MDP that consists of a car placed stochastically
   at the bottom of a sinusoidal valley, with the only possible actions being the accelerations
   that can be applied to the car in either direction. The goal of the MDP is to strategically
   accelerate the car to reach the goal state on top of the right hill. There are two versions
   of the mountain car domain in gym: one with discrete actions and one with continuous.
   This version is the one with discrete actions.
   This MDP first appeared in [Andrew Moore's PhD Thesis (1990)](https://www.cl.cam.ac.uk/techreports/UCAM-CL-TR-209.pdf)
   ```
    @TECHREPORT{Moore90efficientmemory-based,
        author = {Andrew William Moore},
        title = {Efficient Memory-based Learning for Robot Control},
        institution = {University of Cambridge},
        year = {1990}
    }
   ```
   ### Observation Space
   The observation is a `ndarray` with shape `(2,)` where the elements correspond to the following:
   | Num | Observation                          | Min  | Max | Unit         |
   |-----|--------------------------------------|------|-----|--------------|
   | 0   | position of the car along the x-axis | -Inf | Inf | position (m) |
   | 1   | velocity of the car                  | -Inf | Inf | position (m) |
   ### Action Space
   There are 3 discrete deterministic actions:
   | Num | Observation             | Value | Unit         |
   |-----|-------------------------|-------|--------------|
   | 0   | Accelerate to the left  | Inf   | position (m) |
   | 1   | Don't accelerate        | Inf   | position (m) |
   | 2   | Accelerate to the right | Inf   | position (m) |
   ### Transition Dynamics:
   Given an action, the mountain car follows the following transition dynamics:
   *velocity<sub>t+1</sub> = velocity<sub>t</sub> + (action - 1) * force - cos(3 * position<sub>t</sub>) * gravity*
   *position<sub>t+1</sub> = position<sub>t</sub> + velocity<sub>t+1</sub>*
   where force = 0.001 and gravity = 0.0025. The collisions at either end are inelastic with the velocity set to 0
   upon collision with the wall. The position is clipped to the range `[-1.2, 0.6]` and
   velocity is clipped to the range `[-0.07, 0.07]`.
   ### Reward:
   The goal is to reach the flag placed on top of the right hill as quickly as possible, as such the agent is
   penalised with a reward of -1 for each timestep.
   ### Starting State
   The position of the car is assigned a uniform random value in *[-0.6 , -0.4]*.
   The starting velocity of the car is always assigned to 0.
   ### Episode End
   The episode ends if either of the following happens:
   1. Termination: The position of the car is greater than or equal to 0.5 (the goal position on top of the right hill)
   2. Truncation: The length of the episode is 200.
   ### Arguments
   ```
    gym.make('MountainCar-v0')
   ```
   ### Version History
   * v0: Initial versions release (1.0.0)

# ¿Qué es Gym de OpenAI?

Gym es un API desarrollada para resolver problemas de aprendizaje por refuerzo. La intención es tener una forma práctica de representar diversos ambientes de RL (Reinforcement Learning).

Las categorías principales de ambientes que están implementados en Gym son:
* Atari
* MuJoCo
* Toy Text
* Classic Control
* Box2D

Cada una de estas categorías contiene su propio conjunto de ambientes en los que agentes tienen que resolver tareas especificas. La API provee un conjunto de métodos estandarizados para comunicarse con el ambiente y poder controlarlo de manera sencilla con alguna función que establezca la política de acciones para tener éxito en resolver el reto del ambiente.

En nuestro caso, 'MountainCar-v0' es la implementación de un ambiente de 'Classic Control' que puede ser manipulado fácilmente con ayuda de una política. Para ello, cada ambiente debde de ser controlado dentro de un ciclo en donde las interacciones entre el agente y el ambiente pueden ser observadas, al mismo tiempo que las observaciones y recompensas son utilizadas para que el agente tome las acciones para lograr ganar el reto.  

Este ciclo básico de control en `gym` se ve de la siguiente manera.

In [3]:
import gym
import random
import numpy as np

Para instalar gym:\
Utilizar el siguiente comando en pip.\
Ref:https://github.com/openai/gym

In [2]:
#import sys
#!{sys.executable} -m pip install gym[all]

# En caso de solo querer instalar las dependencias de classic control utilizar:
#!{sys.executable} -m pip install gym[classic_control]

In [4]:
env_name = "MountainCar-v0"
env = gym.make(env_name, render_mode='human')
env.action_space.seed(42)
print('action_space.seed(42) = ', env.action_space.seed(42))
observation, info = env.reset(seed=42)

for _ in range(10):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
env.close()

action_space.seed(42) =  [42]


Vamos a analizar lo que hace este simple ejemplo.

Para inicializar un ambiente hay que utilizar el método `gym.make()`. El método recibe como mínimo el argumento del nombre del ambienta que se quiere inicializar. Además, del parametro `render_mode='human'` que constantemente esta generando una imagen del estado del agente y el ambiente, generalmente utilizado para visualización humana (ver como el agente se comporta en la animación gráfica).

Cada ambiente específicael formato de acciones validas a traves del atributo `env.action_space`. Y para definir el formato de observaciones validas con el atributo `env.observation_space`. Por ejemplo, para 'MountainCar-v0' tenemos:

In [5]:
print(f'{env_name + " attributes":-^70}')
print("Observation space: ", env.observation_space)
print("Action space:\t   ", env.action_space)

----------------------MountainCar-v0 attributes-----------------------
Observation space:  Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Action space:	    Discrete(3)


Este ambiente esta definido por un [Space](https://www.gymlibrary.dev/api/spaces/) con un espacio de observación de tipo `Box` y un espacio de acción de tipo `Discrete` (hay distintos tipos descritos en el enlace de Space).
* `Box`: Es un espacio en el que podemos tener los límites superior e inferior que describen los posibles valores que una observación puede tener.
* `Discrete`: Describe un espacio discreto en el que $\{0,1,...,n-1\}$ son los posibles valores de las acciones u observaciones pueden tener.

Antes de utilizar cualquier ambiente, hay que reiniciarlo utilizando el método `env.reset()` que regresa 'observation' y un diccionario 'info'. En nuestro caso, nos concentraremos en el valor de la observación, que será un `numpy.array` con un valor inicial dentro del `observation_space` del ambiente. Por ejemplo:

In [6]:
env_name = "MountainCar-v0"
env = gym.make(env_name, render_mode='human')
env.action_space.seed(42)
observation, info = env.reset()
print("Observation: ", observation)
print("Observation shape: ", observation.shape)
env.close()

Observation:  [-0.4533822  0.       ]
Observation shape:  (2,)


En este caso el valor inicial de la observación contiene un `numpy.array` de forma `(2,)`. En este caso, como esta descrito en el ambiente de [MountainCar-v0](https://www.gymlibrary.dev/environments/classic_control/mountain_car/) (ver la sección 'MountainCar-v0'), este arreglo contiene la posición del agente en el eje-x y la velocidad inicial del agente. Por lo tanto, el estado inicial del agente en el ambiente sería:
* El auto se encuentra en $-0.49841654$ en el eje x.
* La velocidad inicial de auto es de $0$.

Ahora, además de los atributos mencionados y el método de `reset()`, el [Core](https://www.gymlibrary.dev/api/core/) de la API ofrece otros métodos adicionales para poder interactuar con el agente y ambiente instanciados:
* `step()`: Corre un paso de la dinámica del ambiente. Recibe como parámetro una acción del agente y regresa una tupla (observation, reward, terminated, truncated, info).
* `render()`: [Ver render()](https://www.gymlibrary.dev/api/core/)

Entonces, retomando el ejemplo incial:

In [15]:
env_name = "MountainCar-v0" # Definimos nombre del ambiente
env = gym.make(env_name, render_mode='human') # Lo instanciamos
env.action_space.seed(42) # Para garantizar un muestreo aleatorio del espacio de acción.
observation, info = env.reset(seed=42) # Llamamos reset()

for _ in range(10): # Damos 10 pasos
    action = env.action_space.sample() # Hacemos un muestreo del espacio de acción
    observation, reward, terminated, truncated, info = env.step(action) # Damos un paso con la acción seleccionada
    print("Action: ", action)  
    print("Observation: ", observation)
    print("Reward: ", reward)
    print("Terminated: ", terminated)
    print("Truncated: ", truncated)
    print("Info: ", info)
    if terminated or truncated: # Revisamos sí una de las dos variables que terminan el episodio es verdadera
        observation, info = env.reset(seed=42) # Sí una de las dos es verdadera; entonces volvemos a reiniciar el ambiente
env.close()

Action:  0
Observation:  [-0.44679132 -0.00158252]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  2
Observation:  [-0.4479448  -0.00115349]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  1
Observation:  [-0.44966084 -0.00171604]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  1
Observation:  [-0.4519269  -0.00226604]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  1
Observation:  [-0.4547263  -0.00279944]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  2
Observation:  [-0.45703864 -0.00231232]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  0
Observation:  [-0.46084684 -0.00380821]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  2
Observation:  [-0.46412292 -0.00327607]
Reward:  -1.0
Terminated:  False
Truncated:  False
Info:  {}
Action:  0
Observation:  [-0.46884272 -0.00471978]
Reward:  -1.0
Terminated:  False
Truncated:  False
In

Para estos diez pasos cada acción depende del muestreo que hagamos del espacio de acción definido por el agente. En cada paso, vemos que la recompenza es de $-1.0$ y para la bandera de 'terminado' y 'truncado' tenemos el valor `False`. Esto debido a las condiciones para el fin del episiodio (episode end) de 'MountainCar-v0'. 

La intención de `gym` es brindar la posibilidad de utilizar estos métodos del API para poder controlar el agente con alguna política de función para determinar la nueva acción en cada paso. 

Por ejemplo:
```
import gym
env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = policy(observation)  # User-defined policy function
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()
```
En este caso, la variable `action` sería actualizada por una función definida que toma como parametro la observación de cada paso.

Este ejemplo es muy sencillo pero la idea es dar un ejemplo de como se podría utilizar el API de Gym.

## Controlando el coche con las flechas del teclado

Además de poder utilizar el API para solución de problemas de RL. El API también cuenta con métodos adicionales en [Utils](https://www.gymlibrary.dev/api/utils/). como `play`.



In [13]:
import pygame
from gym.utils.play import play
mapping = {(pygame.K_LEFT,): 0, (pygame.K_RIGHT,): 2, (pygame.K_DOWN,):1}
play(gym.make("MountainCar-v0",render_mode='rgb_array'), keys_to_action=mapping)

La utilidad `play` recibe un ambiente con un modo de renderizado y un mapeao de las teclas a utilizar para cada acción. En este caso la flecha izquierda hace que el coche acelere en esa dirección con el valor de $0$; igualmente para las demás acciones $2$ hacia la derecha y $1$ para no aceleración. 